In [2]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# Step 1: Load the Dataset
data = pd.read_csv(r"C:\Users\amrut\OneDrive\Documents\indian_food.csv")

# Display the first few rows of the dataset
print(data.head())

# Step 2: Data Preprocessing
# Convert categorical columns into numerical values using LabelEncoder
label_encoders = {}
for column in ['name', 'ingredients', 'diet', 'flavor_profile', 'course', 'state', 'region']:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

# Convert 'prep_time' and 'cook_time' to numeric by extracting numerical values
data['prep_time'] = data['prep_time'].str.extract(r'(\d+)').astype(float)
data['cook_time'] = data['cook_time'].str.extract(r'(\d+)').astype(float)

# Handle missing or NaN values if present
data.fillna(0, inplace=True)

# Assuming the calories are in a column called 'Calories'
# If calories aren't available, you might need to map them manually
if 'Calories' in data.columns:
    target = data['Calories']
    features = data.drop('Calories', axis=1)
else:
    print("Calorie column not found in the dataset")
    target = np.zeros(len(data))  # Placeholder target
    features = data

# Normalize numerical data using StandardScaler
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Step 3: Split the Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Step 4: Build the Neural Network
model = Sequential()

# Input layer and hidden layers
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))

# Output layer for regression task (calorie prediction)
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Step 5: Train the Model
# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping], verbose=1)

# Step 6: Evaluate the Model
# Evaluate the model on test data
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Mean Absolute Error on Test Data: {mae}")

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error for test data
mse = np.mean((y_pred - y_test)**2)
print(f"Mean Squared Error: {mse}")

# Step 7: Visualize Training and Validation Performance
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['mean_absolute_error'], label='Training MAE')
plt.plot(history.history['val_mean_absolute_error'], label='Validation MAE')
plt.title('Mean Absolute Error Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


             name                                        ingredients  \
0      Balu shahi                    Maida flour, yogurt, oil, sugar   
1          Boondi                            Gram flour, ghee, sugar   
2  Gajar ka halwa       Carrots, milk, sugar, ghee, cashews, raisins   
3          Ghevar  Flour, ghee, kewra, milk, clarified butter, su...   
4     Gulab jamun  Milk powder, plain flour, baking powder, ghee,...   

         diet  prep_time  cook_time flavor_profile   course        state  \
0  vegetarian         45         25          sweet  dessert  West Bengal   
1  vegetarian         80         30          sweet  dessert    Rajasthan   
2  vegetarian         15         60          sweet  dessert       Punjab   
3  vegetarian         15         30          sweet  dessert    Rajasthan   
4  vegetarian         15         40          sweet  dessert  West Bengal   

  region  
0   East  
1   West  
2  North  
3   West  
4   East  


AttributeError: Can only use .str accessor with string values!